In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

In [3]:
import kagglehub

# Download the dataset to the specified path
path = kagglehub.dataset_download("jp797498e/twitter-entity-sentiment-analysis")

print("Path to dataset files:", path)

/home/adityadev/.pyenv/versions/3.10.12/envs/GPTDecoder/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/adityadev/.cache/kagglehub/datasets/jp797498e/twitter-entity-sentiment-analysis/versions/2


In [4]:
import pandas as pd

train_df = pd.read_csv(f"{path}/twitter_training.csv", header = None)
val_df = pd.read_csv(f"{path}/twitter_validation.csv", header = None)

In [5]:
train_df.head()

,0,1,2,3
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [6]:
from data_loader import CustomDataLoader

text_col = 3    # Column index for input text
label_col = 2   # Column index for labels
batch_size = 8 # Batch size for training and validation

# Initialize CustomDataLoader
custom_loader = CustomDataLoader(train_df, val_df, text_col, label_col, batch_size)

# Get train and validation loaders
train_loader = custom_loader.get_train_loader()
val_loader = custom_loader.get_val_loader()

In [7]:
# #Accessing random batches for training
# train_iter = iter(train_loader)
# Xb, Yb, mask = next(train_iter)  # Get one random batch
# print("Tokens:", Xb.shape)  # (batch_size, block_size)
# print("Labels:", Yb.shape)  # (batch_size,)
# print("Mask:", mask.shape)  # (batch_size, block_size)

In [8]:
model_params = {
    "vocab_size": train_loader.dataset.vocab_size,
    "num_embeddings": 64,
    "block_size": train_loader.dataset.block_size,
    "num_heads": 4,
    "num_layers": 4,
    "output_classes": len(train_loader.dataset.labels_lookup_dict),
    "dropout": 0.2,
    "device": 'cuda' if torch.cuda.is_available() else 'cpu'
}

In [9]:
train_params = {
    "num_epochs": 100,
    "eval_interval": 10,
    "eval_iters": 10,
    "learning_rate": 0.01
}

In [10]:
save_models_path = "./models_v1"

In [13]:
from train import TrainingLoop

TrainingLoop(model_params, train_params).train(train_loader, val_loader, save_models_path, resume_path="/home/adityadev/GPTDecoder/models_v1/best_model.pth")

/home/adityadev/GPTDecoder/train.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(load_path, map_location=self.device)  # Ensure checkpoint is lo